<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Breast_Diagnosis_Model_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tensorflow-addons

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import pandas as pd
import seaborn as sns

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define your data directory path and resolutions
data_dir = '/content/drive/My Drive/data2'
resolutions = [(30, 30), (70, 70), (100, 100)]

# Get class names
classes = [cls for cls in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, cls))]

# Simplified CNN model
def build_compile_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Plot confusion matrices
def plot_confusion_matrices(cm_train, cm_test, classes, title):
    fig, ax = plt.subplots(1, 2, figsize=(10, 6))

    sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 12}, ax=ax[0])
    ax[0].set_title(f'Train {title} CM')
    ax[0].set_xlabel('Predicted label')
    ax[0].set_ylabel('True label')
    ax[0].set_xticks(np.arange(len(classes)) + 0.5)
    ax[0].set_xticklabels(classes, rotation=45)
    ax[0].set_yticks(np.arange(len(classes)) + 0.5)
    ax[0].set_yticklabels(classes, rotation=0)

    sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', annot_kws={"size": 12}, ax=ax[1])
    ax[1].set_title(f'Test {title} CM')
    ax[1].set_xlabel('Predicted label')
    ax[1].set_ylabel('True label')
    ax[1].set_xticks(np.arange(len(classes)) + 0.5)
    ax[1].set_xticklabels(classes, rotation=45)
    ax[1].set_yticks(np.arange(len(classes)) + 0.5)
    ax[1].set_yticklabels(classes, rotation=0)

    plt.tight_layout()
    plt.show()

# Plot convergence curves
def plot_convergence_curves(history, resolution):
    plt.figure(figsize=(10, 5))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Accuracy Convergence (Resolution {resolution})')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Loss Convergence (Resolution {resolution})')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

# Function to preprocess and generate data in batches
def data_generator(data_dir, resolution, classes, batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=resolution,
        batch_size=batch_size,
        class_mode='sparse',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=resolution,
        batch_size=batch_size,
        class_mode='sparse',
        subset='validation'
    )

    return train_generator, validation_generator

# Train models and collect metrics
metrics = []

for res in resolutions:
    train_generator, validation_generator = data_generator(data_dir, res, classes)

    model = build_compile_model((res[0], res[1], 3), len(classes))

    # Save model architecture plot
    plot_model(model, to_file=f'model_architecture_{res[0]}x{res[1]}.png', show_shapes=True, show_layer_names=True)

    history = model.fit(train_generator, epochs=15, validation_data=validation_generator, verbose=1)

    # Collect train and test data
    y_train = train_generator.classes
    y_test = validation_generator.classes
    y_pred_train = np.argmax(model.predict(train_generator), axis=1)
    y_pred_test = np.argmax(model.predict(validation_generator), axis=1)

    cm_train = confusion_matrix(y_train, y_pred_train)
    cm_test = confusion_matrix(y_test, y_pred_test)
    report = classification_report(y_test, y_pred_test, target_names=classes, output_dict=True)

    metrics.append({
        'Resolution': f'{res[0]}x{res[1]}',
        'Test Accuracy': history.history['val_accuracy'][-1],
        'Test Precision': report['macro avg']['precision'],
        'Test Recall': report['macro avg']['recall'],
        'Test F1-Score': report['macro avg']['f1-score']
    })

    # Print confusion matrix and plot convergence curves
    plot_confusion_matrices(cm_train, cm_test, classes, f'(Resolution {res})')
    plot_convergence_curves(history, res)

# Plot Boxplot
def plot_boxplot(metrics):
    melted_df = pd.melt(pd.DataFrame(metrics), id_vars='Resolution', var_name='Metric', value_name='Score')
    plt.figure(figsize=(10, 8))
    sns.boxplot(x='Resolution', y='Score', hue='Metric', data=melted_df)
    plt.title('Performance Metrics Boxplot')
    plt.savefig('boxplot_curves.png')
    plt.show()

# Plot Boxplot of metrics
plot_boxplot(metrics)
